In [1]:
import scipy as sp 
from matplotlib import pyplot as plt
import cupy as cp
from cupy.linalg import eigh

from tqdm import *

import pennylane as qml
import pennylane.numpy as np 

In [2]:
n = 8
n_wires = n
dev = qml.device("lightning.gpu", wires=n_wires)

In [3]:
def generate_Heisenberg_hamiltonian(n):
    """
    Generating Hamiltonian of n-qubit Heisenberg model.

    Args:
        - n (int): The number of qubits 

    Returns:
        - cost_h (qml.Hamiltonian): the hamiltonian of n-qubit Heisenberg model
    """
    coeffs = [1]*3*(n-1)
    obs = []
    for i in range(n-1):
        obs.append(qml.PauliX(i) @ qml.PauliX(i+1) )
        obs.append(qml.PauliY(i) @ qml.PauliY(i+1) )
        obs.append(qml.PauliZ(i) @ qml.PauliZ(i+1) )
    cost_h = qml.Hamiltonian(coeffs, obs)
    return cost_h

In [4]:
def ground_energy(n):
    """ 
    Computing the ground state energy (minimum eigenvalue) of the n-qubit Heisenberg model using CuPy.
    
    Args:
        - n (int): The number of qubits. 

    Returns:
        - min_energy (floats): Minimum eigenvalue of the Heisenberg model.
    
    """
    hamiltonian = generate_Heisenberg_hamiltonian(n)
    hamiltonian_matrix = qml.matrix(hamiltonian)
    h_matrix_cupy = cp.array(hamiltonian_matrix)
    eigv,eigc = eigh(h_matrix_cupy)

    return eigv[0]
    

In [11]:
min_energies = []
for i in trange(4,16):
    min_energy = ground_energy(i)
    min_energies.append(min_energy)
print(min_energies)

100%|██████████| 12/12 [02:07<00:00, 10.64s/it]


[array(-6.46410162), array(-7.71154501), array(-9.97430854), array(-11.34495872), array(-13.49973039), array(-14.94528683), array(-17.03214083), array(-18.52837321), array(-20.56836253), array(-22.10128839), array(-24.10689865), array(-25.66768197)]


In [12]:
min_energies

[array(-6.46410162),
 array(-7.71154501),
 array(-9.97430854),
 array(-11.34495872),
 array(-13.49973039),
 array(-14.94528683),
 array(-17.03214083),
 array(-18.52837321),
 array(-20.56836253),
 array(-22.10128839),
 array(-24.10689865),
 array(-25.66768197)]

In [97]:
def qnn_model(params,depth,cost_h):
    def U3_layer(params):
        for i in range(n_wires):
            qml.U3(params[3*i],params[3*i+1],params[3*i+2],wires=i)
    
    def IsingXYZ_layer(params):
        for i in range(n_wires-1):
            qml.IsingXY(params[3*i],wires=[i,(i+1)%n_wires])
            qml.IsingXY(params[3*i+1],wires=[i,(i+1)%n_wires])
            qml.IsingZZ(params[3*i+2],wires=[i,(i+1)%n_wires])
            pass

    def U3_IsingXYZ(params):
        U3_layer(params[0])
        IsingXYZ_layer(params[1])

    qml.layer(U3_IsingXYZ,depth,params[:-1])
    
    return qml.expval(cost_h)

In [102]:
n = 4
depth = 4
n_wires = n
dev = qml.device('default.qubit', wires=n_wires)
cost_h = generate_Heisenberg_hamiltonian(n)

@qml.qnode(dev)
def cost_fn(params):
    return qnn_model(params,depth,cost_h)

optimizer = qml.AdamOptimizer(stepsize=0.2)
init_params = np.random.uniform(-1/depth*np.pi,1/depth*np.pi,size = (depth+1,2,n_wires*3))
steps = 1000
params = init_params
best_params = params
best_cost = 0
for i in range(steps):
        params = optimizer.step(cost_fn, params)
        cost_tmp = cost_fn(params)
        if(cost_tmp < best_cost):
                best_cost =cost_tmp
                best_params = params
        if( i % 100 == 0):
            print(
                    "Iter: {:5d} | Cost: {:0.10f} "
                    "".format(i + 1, cost_fn(params)))
print("best_cost = ",best_cost)

Iter:     1 | Cost: -1.9064569029 
Iter:   101 | Cost: -6.4640025815 
Iter:   201 | Cost: -6.4624842925 
Iter:   301 | Cost: -6.4303568797 
Iter:   401 | Cost: -6.4548905045 
Iter:   501 | Cost: -6.4614619469 
Iter:   601 | Cost: -6.4485568657 
Iter:   701 | Cost: -6.4577642811 
Iter:   801 | Cost: -6.4059367165 
Iter:   901 | Cost: -6.4051266748 
best_cost =  -6.464099028748394


In [101]:
n = 5
depth = 4
n_wires = n
dev = qml.device('default.qubit', wires=n_wires)
cost_h = generate_Heisenberg_hamiltonian(n)

@qml.qnode(dev)
def cost_fn(params):
    return qnn_model(params,depth,cost_h)

optimizer = qml.AdamOptimizer(stepsize=0.2)
init_params = np.random.uniform(-1/depth*np.pi,1/depth*np.pi,size = (depth+1,2,n_wires*3))
steps = 1000
params = init_params
best_params = params
best_cost = 0
for i in range(steps):
        params = optimizer.step(cost_fn, params)
        cost_tmp = cost_fn(params)
        if(cost_tmp < best_cost):
                best_cost =cost_tmp
                best_params = params
        if( i % 100 == 0):
            print(
                    "Iter: {:5d} | Cost: {:0.10f} "
                    "".format(i + 1, cost_fn(params)))
print("best_cost = ",best_cost)

Iter:     1 | Cost: -1.2490150850 
Iter:   101 | Cost: -7.6994295353 
Iter:   201 | Cost: -7.7031429422 
Iter:   301 | Cost: -7.7034338988 
Iter:   401 | Cost: -7.7055427796 
Iter:   501 | Cost: -7.7084903935 
Iter:   601 | Cost: -7.7072087108 
Iter:   701 | Cost: -7.6401847383 
Iter:   801 | Cost: -7.6969825252 
Iter:   901 | Cost: -7.6876499504 
best_cost =  -7.7096774160358095


In [99]:
n = 6
depth = 4
n_wires = n
dev = qml.device('default.qubit', wires=n_wires)
cost_h = generate_Heisenberg_hamiltonian(n)

@qml.qnode(dev)
def cost_fn(params):
    return qnn_model(params,depth,cost_h)

optimizer = qml.AdamOptimizer(stepsize=0.1)
init_params = np.random.uniform(-1/depth*np.pi,1/depth*np.pi,size = (depth+1,2,n_wires*3))
steps = 1000
params = init_params
best_params = params
best_cost = 0
for i in range(steps):
        params = optimizer.step(cost_fn, params)
        cost_tmp = cost_fn(params)
        if(cost_tmp < best_cost):
                best_cost =cost_tmp
                best_params = params
        if( i % 100 == 0):
            print(
                    "Iter: {:5d} | Cost: {:0.10f} "
                    "".format(i + 1, cost_fn(params)))
print("best_cost = ",best_cost)

Iter:     1 | Cost: -1.6865818539 
Iter:   101 | Cost: -9.9520136980 
Iter:   201 | Cost: -9.9713003644 
Iter:   301 | Cost: -9.9735023872 
Iter:   401 | Cost: -9.9720093550 
Iter:   501 | Cost: -9.9624799837 
Iter:   601 | Cost: -9.9673426227 
Iter:   701 | Cost: -9.9661537660 
Iter:   801 | Cost: -9.9726984797 
Iter:   901 | Cost: -9.9706013400 
best_cost =  -9.974049496422639


In [96]:
n = 7
depth = 4
n_wires = n
dev = qml.device('default.qubit', wires=n_wires)
cost_h = generate_Heisenberg_hamiltonian(n)

@qml.qnode(dev)
def cost_fn(params):
    return qnn_model(params,depth,cost_h)

optimizer = qml.AdamOptimizer(stepsize=0.1)
init_params = np.random.uniform(-1/depth*np.pi,1/depth*np.pi,size = (depth+1,2,n_wires*3))
steps = 1000
params = init_params
best_params = params
best_cost = 0
for i in range(steps):
        params = optimizer.step(cost_fn, params)
        cost_tmp = cost_fn(params)
        if(cost_tmp < best_cost):
                best_cost =cost_tmp
                best_params = params
        if( i % 100 == 0):
            print(
                    "Iter: {:5d} | Cost: {:0.10f} "
                    "".format(i + 1, cost_fn(params)))
print("best_cost = ",best_cost)

Iter:     1 | Cost: -0.9841177870 
Iter:   101 | Cost: -11.1959838567 
Iter:   201 | Cost: -11.3079321429 
Iter:   301 | Cost: -11.3021551945 
Iter:   401 | Cost: -11.2878516937 
Iter:   501 | Cost: -11.3267215099 
Iter:   601 | Cost: -11.3335424091 
Iter:   701 | Cost: -11.3308169365 
Iter:   801 | Cost: -11.3328468215 
Iter:   901 | Cost: -11.2861402997 
best_cost =  -11.336856144837238


In [94]:
n = 8
depth = 4
n_wires = n
dev = qml.device('default.qubit', wires=n_wires)
cost_h = generate_Heisenberg_hamiltonian(n)

@qml.qnode(dev)
def cost_fn(params):
    return qnn_model(params,depth,cost_h)

optimizer = qml.AdamOptimizer(stepsize=0.1)
init_params = np.random.uniform(-1/depth*np.pi,1/depth*np.pi,size = (depth+1,2,n_wires*3))
steps = 1000
params = init_params
best_params = params
best_cost = 0
for i in range(steps):
        params = optimizer.step(cost_fn, params)
        cost_tmp = cost_fn(params)
        if(cost_tmp < best_cost):
                best_cost =cost_tmp
                best_params = params
        if( i % 100 == 0):
            print(
                    "Iter: {:5d} | Cost: {:0.10f} "
                    "".format(i + 1, cost_fn(params)))
print("best_cost = ",best_cost)

Iter:     1 | Cost: 0.9630336179 
Iter:   101 | Cost: -13.4498684021 
Iter:   201 | Cost: -13.4606706574 
Iter:   301 | Cost: -13.4751287748 
Iter:   401 | Cost: -13.4362727901 
Iter:   501 | Cost: -13.4868449423 
Iter:   601 | Cost: -13.4867337679 
Iter:   701 | Cost: -13.4617123778 
Iter:   801 | Cost: -13.4825689915 
Iter:   901 | Cost: -13.4911436552 
best_cost =  -13.492362472499543


In [104]:
n = 9
depth = 9
n_wires = n
dev = qml.device('default.qubit', wires=n_wires)
cost_h = generate_Heisenberg_hamiltonian(n)

@qml.qnode(dev)
def cost_fn(params):
    return qnn_model(params,depth,cost_h)

optimizer = qml.AdamOptimizer(stepsize=0.1)
init_params = np.random.uniform(-1/depth*np.pi,1/depth*np.pi,size = (depth+1,2,n_wires*3))
steps = 1000
params = init_params
best_params = params
best_cost = 0
for i in range(steps):
        params = optimizer.step(cost_fn, params)
        cost_tmp = cost_fn(params)
        if(cost_tmp < best_cost):
                best_cost =cost_tmp
                best_params = params
        if( i % 100 == 0):
            print(
                    "Iter: {:5d} | Cost: {:0.10f} "
                    "".format(i + 1, cost_fn(params)))
print("best_cost = ",best_cost)

Iter:     1 | Cost: -1.3199894221 
Iter:   101 | Cost: -14.9260161960 
Iter:   201 | Cost: -14.9229251786 
Iter:   301 | Cost: -14.9256587373 
Iter:   401 | Cost: -14.9345237969 
Iter:   501 | Cost: -14.9259298616 
Iter:   601 | Cost: -14.9368419352 
Iter:   701 | Cost: -14.9188465412 
Iter:   801 | Cost: -14.9110912926 
Iter:   901 | Cost: -14.9176950578 
best_cost =  -14.940541397947392


In [105]:
n = 10
depth = 10
n_wires = n
dev = qml.device('default.qubit', wires=n_wires)
cost_h = generate_Heisenberg_hamiltonian(n)

@qml.qnode(dev)
def cost_fn(params):
    return qnn_model(params,depth,cost_h)

optimizer = qml.AdamOptimizer(stepsize=0.1)
init_params = np.random.uniform(-1/depth*np.pi,1/depth*np.pi,size = (depth+1,2,n_wires*3))
steps = 1000
params = init_params
best_params = params
best_cost = 0
for i in range(steps):
        params = optimizer.step(cost_fn, params)
        cost_tmp = cost_fn(params)
        if(cost_tmp < best_cost):
                best_cost =cost_tmp
                best_params = params
        if( i % 100 == 0):
            print(
                    "Iter: {:5d} | Cost: {:0.10f} "
                    "".format(i + 1, cost_fn(params)))
print("best_cost = ",best_cost)

Iter:     1 | Cost: 2.5627972491 
Iter:   101 | Cost: -16.9860244465 
Iter:   201 | Cost: -17.0081167197 
Iter:   301 | Cost: -16.9830253385 
Iter:   401 | Cost: -17.0031120662 
Iter:   501 | Cost: -17.0123298597 
Iter:   601 | Cost: -17.0156906449 
Iter:   701 | Cost: -16.9960463697 
Iter:   801 | Cost: -17.0221588753 
Iter:   901 | Cost: -17.0216445438 
best_cost =  -17.027377362167197


In [106]:
n = 11
depth = 11
n_wires = n
dev = qml.device('default.qubit', wires=n_wires)
cost_h = generate_Heisenberg_hamiltonian(n)

@qml.qnode(dev)
def cost_fn(params):
    return qnn_model(params,depth,cost_h)

optimizer = qml.AdamOptimizer(stepsize=0.1)
init_params = np.random.uniform(-1/depth*np.pi,1/depth*np.pi,size = (depth+1,2,n_wires*3))
steps = 1000
params = init_params
best_params = params
best_cost = 0
for i in range(steps):
        params = optimizer.step(cost_fn, params)
        cost_tmp = cost_fn(params)
        if(cost_tmp < best_cost):
                best_cost =cost_tmp
                best_params = params
        if( i % 100 == 0):
            print(
                    "Iter: {:5d} | Cost: {:0.10f} "
                    "".format(i + 1, cost_fn(params)))
print("best_cost = ",best_cost)

Iter:     1 | Cost: 2.3531702521 
Iter:   101 | Cost: -18.4037190016 
Iter:   201 | Cost: -18.3881452777 
Iter:   301 | Cost: -18.4273047162 
Iter:   401 | Cost: -18.4841174126 
Iter:   501 | Cost: -18.4680682321 
Iter:   601 | Cost: -18.4206705532 
Iter:   701 | Cost: -18.5053437965 
Iter:   801 | Cost: -18.4857840858 
Iter:   901 | Cost: -18.4938752980 
best_cost =  -18.515710073693807


In [110]:
n = 12
depth = 12
n_wires = n
dev = qml.device('default.qubit', wires=n_wires)
cost_h = generate_Heisenberg_hamiltonian(n)

@qml.qnode(dev)
def cost_fn(params):
    return qnn_model(params,depth,cost_h)

optimizer = qml.AdamOptimizer(stepsize=0.1)
init_params = np.random.uniform(-1/depth*np.pi,1/depth*np.pi,size = (depth+1,2,n_wires*3))
steps = 1000
params = init_params
best_params = params
best_cost = 0
for i in range(steps):
        params = optimizer.step(cost_fn, params)
        cost_tmp = cost_fn(params)
        if(cost_tmp < best_cost):
                best_cost =cost_tmp
                best_params = params
        if( i % 100 == 0):
            print(
                    "Iter: {:5d} | Cost: {:0.10f} "
                    "".format(i + 1, cost_fn(params)))
print("best_cost = ",best_cost)

Iter:     1 | Cost: 2.0700028342 
Iter:   101 | Cost: -20.4920779478 
Iter:   201 | Cost: -20.4449952859 
Iter:   301 | Cost: -20.4347338077 
Iter:   401 | Cost: -20.4990702647 
Iter:   501 | Cost: -20.5184434123 
Iter:   601 | Cost: -20.5369878104 
Iter:   701 | Cost: -20.5130054443 
Iter:   801 | Cost: -20.5442340016 
Iter:   901 | Cost: -20.5265837674 
best_cost =  -20.550576406512622


In [109]:
n = 13
depth = 13
n_wires = n
dev = qml.device('default.qubit', wires=n_wires)
cost_h = generate_Heisenberg_hamiltonian(n)

@qml.qnode(dev)
def cost_fn(params):
    return qnn_model(params,depth,cost_h)

optimizer = qml.AdamOptimizer(stepsize=0.1)
init_params = np.random.uniform(-1/depth*np.pi,1/depth*np.pi,size = (depth+1,2,n_wires*3))
steps = 1000
params = init_params
best_params = params
best_cost = 0
for i in range(steps):
        params = optimizer.step(cost_fn, params)
        cost_tmp = cost_fn(params)
        if(cost_tmp < best_cost):
                best_cost =cost_tmp
                best_params = params
        if( i % 100 == 0):
            print(
                    "Iter: {:5d} | Cost: {:0.10f} "
                    "".format(i + 1, cost_fn(params)))
print("best_cost = ",best_cost)

Iter:     1 | Cost: 3.2515176568 
Iter:   101 | Cost: -21.8844473165 
Iter:   201 | Cost: -21.9961541904 
Iter:   301 | Cost: -21.7070648817 
Iter:   401 | Cost: -21.9980377044 
Iter:   501 | Cost: -21.9474763572 
Iter:   601 | Cost: -22.0117437722 
Iter:   701 | Cost: -21.9694764777 
Iter:   801 | Cost: -21.9481724924 
Iter:   901 | Cost: -22.0597729929 
best_cost =  -22.079562924658916


In [108]:
n = 14
depth = 14
n_wires = n
dev = qml.device('default.qubit', wires=n_wires)
cost_h = generate_Heisenberg_hamiltonian(n)

@qml.qnode(dev)
def cost_fn(params):
    return qnn_model(params,depth,cost_h)

optimizer = qml.AdamOptimizer(stepsize=0.1)
init_params = np.random.uniform(-1/depth*np.pi,1/depth*np.pi,size = (depth+1,2,n_wires*3))
steps = 1000
params = init_params
best_params = params
best_cost = 0
for i in range(steps):
        params = optimizer.step(cost_fn, params)
        cost_tmp = cost_fn(params)
        if(cost_tmp < best_cost):
                best_cost =cost_tmp
                best_params = params
        if( i % 100 == 0):
            print(
                    "Iter: {:5d} | Cost: {:0.10f} "
                    "".format(i + 1, cost_fn(params)))
print("best_cost = ",best_cost)

Iter:     1 | Cost: 4.9820427210 
Iter:   101 | Cost: -23.8173749081 
Iter:   201 | Cost: -23.8961347870 
Iter:   301 | Cost: -24.0197555689 
Iter:   401 | Cost: -23.9550081530 
Iter:   501 | Cost: -24.0088101808 
Iter:   601 | Cost: -24.0484891732 
Iter:   701 | Cost: -23.9856505695 
Iter:   801 | Cost: -24.0215152690 
Iter:   901 | Cost: -24.0077276500 
best_cost =  -24.062238619518762


In [107]:
n = 15
depth = 15
n_wires = n
dev = qml.device('default.qubit', wires=n_wires)
cost_h = generate_Heisenberg_hamiltonian(n)

@qml.qnode(dev)
def cost_fn(params):
    return qnn_model(params,depth,cost_h)

optimizer = qml.AdamOptimizer(stepsize=0.1)
init_params = np.random.uniform(-1/depth*np.pi,1/depth*np.pi,size = (depth+1,2,n_wires*3))
steps = 1000
params = init_params
best_params = params
best_cost = 0
for i in range(steps):
        params = optimizer.step(cost_fn, params)
        cost_tmp = cost_fn(params)
        if(cost_tmp < best_cost):
                best_cost =cost_tmp
                best_params = params
        if( i % 100 == 0):
            print(
                    "Iter: {:5d} | Cost: {:0.10f} "
                    "".format(i + 1, cost_fn(params)))
print("best_cost = ",best_cost)

Iter:     1 | Cost: 2.5235741822 
Iter:   101 | Cost: -25.4827514979 
Iter:   201 | Cost: -25.5254218507 
Iter:   301 | Cost: -25.5328326226 
Iter:   401 | Cost: -25.5478371355 
Iter:   501 | Cost: -25.5879956744 
Iter:   601 | Cost: -25.5363103290 
Iter:   701 | Cost: -25.5644183163 
Iter:   801 | Cost: -25.5378510793 
Iter:   901 | Cost: -25.4941571132 
best_cost =  -25.6185560845312


In [ ]:
n = 15
depth = 15
n_wires = n
dev = qml.device('default.qubit', wires=n_wires)
cost_h = generate_Heisenberg_hamiltonian(n)

@qml.qnode(dev)
def cost_fn(params):
    return qnn_model(params,depth,cost_h)

optimizer = qml.AdamOptimizer(stepsize=0.1)
init_params = np.random.uniform(-1/depth*np.pi,1/depth*np.pi,size = (depth+1,2,n_wires*3))
steps = 1000
params = init_params
best_params = params
best_cost = 0
for i in range(steps):
        params = optimizer.step(cost_fn, params)
        cost_tmp = cost_fn(params)
        if(cost_tmp < best_cost):
                best_cost =cost_tmp
                best_params = params
        if( i % 100 == 0):
            print(
                    "Iter: {:5d} | Cost: {:0.10f} "
                    "".format(i + 1, cost_fn(params)))
print("best_cost = ",best_cost)